***GENERATED CODE FOR strokeclass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Patient Name', 'transformation_label': 'String Indexer'}], 'feature': 'Patient Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Aarna Mann', 'max': 'Zoya Bora', 'missing': '0', 'distinct': '499'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Patient Name'}, {'feature_label': 'Patient Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Patient Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Marital Status', 'transformation_label': 'String Indexer'}], 'feature': 'Marital Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Divorced', 'max': 'Single', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Marital Status'}, {'feature_label': 'Marital Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Marital Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Work Type', 'transformation_label': 'String Indexer'}], 'feature': 'Work Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Government Job', 'max': 'Self-employed', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Work Type'}, {'feature_label': 'Work Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Work Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Residence Type', 'transformation_label': 'String Indexer'}], 'feature': 'Residence Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Rural', 'max': 'Urban', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Residence Type'}, {'feature_label': 'Residence Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Residence Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Smoking Status', 'transformation_label': 'String Indexer'}], 'feature': 'Smoking Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Currently Smokes', 'max': 'Non-smoker', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Smoking Status'}, {'feature_label': 'Smoking Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Smoking Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alcohol Intake', 'transformation_label': 'String Indexer'}], 'feature': 'Alcohol Intake', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Frequent Drinker', 'max': 'Social Drinker', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alcohol Intake'}, {'feature_label': 'Alcohol Intake', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Alcohol Intake')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Physical Activity', 'transformation_label': 'String Indexer'}], 'feature': 'Physical Activity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Moderate', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Physical Activity'}, {'feature_label': 'Physical Activity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Physical Activity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Family History of Stroke', 'transformation_label': 'String Indexer'}], 'feature': 'Family History of Stroke', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Family History of Stroke'}, {'feature_label': 'Family History of Stroke', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Family History of Stroke')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dietary Habits', 'transformation_label': 'String Indexer'}], 'feature': 'Dietary Habits', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Gluten-Free', 'max': 'Vegetarian', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dietary Habits'}, {'feature_label': 'Dietary Habits', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dietary Habits')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Blood Pressure Levels', 'transformation_label': 'String Indexer'}], 'feature': 'Blood Pressure Levels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '100/101', 'max': '99/92', 'missing': '0', 'distinct': '469'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Blood Pressure Levels'}, {'feature_label': 'Blood Pressure Levels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Blood Pressure Levels')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Cholesterol Levels', 'transformation_label': 'String Indexer'}], 'feature': 'Cholesterol Levels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'HDL: 30, LDL: 102', 'max': 'HDL: 80, LDL: 98', 'missing': '0', 'distinct': '481'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Cholesterol Levels'}, {'feature_label': 'Cholesterol Levels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Cholesterol Levels')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Symptoms', 'transformation_label': 'String Indexer'}], 'feature': 'Symptoms', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '426', 'mean': '', 'stddev': '', 'min': 'Blurred Vision', 'max': 'Weakness, Weakness, Loss of Balance, Headache, Difficulty Speaking', 'missing': '74', 'distinct': '321'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Symptoms'}, {'feature_label': 'Symptoms', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Symptoms')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diagnosis', 'transformation_label': 'String Indexer'}], 'feature': 'Diagnosis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No Stroke', 'max': 'Stroke', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Diagnosis'}, {'feature_label': 'Diagnosis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Diagnosis')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run strokeclassHooks.ipynb
try:
	#sourcePreExecutionHook()

	strokepredictiondataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/stroke_prediction_dataset.csv', 'filename': 'stroke_prediction_dataset.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run strokeclassHooks.ipynb
try:
	#transformationPreExecutionHook()

	strokeclassautofe = TransformationMain.run(strokepredictiondataset,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Patient ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "48753.33", "stddev": "29774.11", "min": "19", "max": "99233", "missing": "0"}, "updatedLabel": "Patient ID"}, {"transformationsData": [{"feature_label": "Patient Name", "transformation_label": "String Indexer"}], "feature": "Patient Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aarna Mann", "max": "Zoya Bora", "missing": "0", "distinct": "499"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Patient Name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "55.29", "stddev": "20.71", "min": "18", "max": "90", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Hypertension", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.29", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Hypertension"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Heart Disease", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.48", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Heart Disease"}, {"transformationsData": [{"feature_label": "Marital Status", "transformation_label": "String Indexer"}], "feature": "Marital Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Divorced", "max": "Single", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Marital Status"}, {"transformationsData": [{"feature_label": "Work Type", "transformation_label": "String Indexer"}], "feature": "Work Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Government Job", "max": "Self-employed", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Work Type"}, {"transformationsData": [{"feature_label": "Residence Type", "transformation_label": "String Indexer"}], "feature": "Residence Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Rural", "max": "Urban", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Residence Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Average Glucose Level", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "128.56", "stddev": "40.42", "min": "60.03", "max": "199.82", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Average Glucose Level"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Body Mass Index (BMI)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "27.29", "stddev": "7.17", "min": "15.03", "max": "39.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Body Mass Index (BMI)"}, {"transformationsData": [{"feature_label": "Smoking Status", "transformation_label": "String Indexer"}], "feature": "Smoking Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Currently Smokes", "max": "Non-smoker", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Smoking Status"}, {"transformationsData": [{"feature_label": "Alcohol Intake", "transformation_label": "String Indexer"}], "feature": "Alcohol Intake", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Frequent Drinker", "max": "Social Drinker", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alcohol Intake"}, {"transformationsData": [{"feature_label": "Physical Activity", "transformation_label": "String Indexer"}], "feature": "Physical Activity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Moderate", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Physical Activity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Stroke History", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.51", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Stroke History"}, {"transformationsData": [{"feature_label": "Family History of Stroke", "transformation_label": "String Indexer"}], "feature": "Family History of Stroke", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Family History of Stroke"}, {"transformationsData": [{"feature_label": "Dietary Habits", "transformation_label": "String Indexer"}], "feature": "Dietary Habits", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Gluten-Free", "max": "Vegetarian", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dietary Habits"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Stress Levels", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.1", "stddev": "2.81", "min": "0.03", "max": "9.96", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Stress Levels"}, {"transformationsData": [{"feature_label": "Blood Pressure Levels", "transformation_label": "String Indexer"}], "feature": "Blood Pressure Levels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "100/101", "max": "99/92", "missing": "0", "distinct": "469"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Blood Pressure Levels"}, {"transformationsData": [{"feature_label": "Cholesterol Levels", "transformation_label": "String Indexer"}], "feature": "Cholesterol Levels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "HDL: 30, LDL: 102", "max": "HDL: 80, LDL: 98", "missing": "0", "distinct": "481"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Cholesterol Levels"}, {"transformationsData": [{"feature_label": "Symptoms", "transformation_label": "String Indexer"}], "feature": "Symptoms", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "426", "mean": "", "stddev": "", "min": "Blurred Vision", "max": "Weakness, Weakness, Loss of Balance, Headache, Difficulty Speaking", "missing": "74", "distinct": "321"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Symptoms"}, {"transformationsData": [{"feature_label": "Diagnosis", "transformation_label": "String Indexer"}], "feature": "Diagnosis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No Stroke", "max": "Stroke", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Diagnosis"}]}))

	#transformationPostExecutionHook(strokeclassautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run strokeclassHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(strokeclassautofe, ["Patient ID", "Age", "Hypertension", "Heart Disease", "Average Glucose Level", "Body Mass Index (BMI)", "Stroke History", "Stress Levels", "Patient Name_stringindexer", "Gender_stringindexer", "Marital Status_stringindexer", "Work Type_stringindexer", "Residence Type_stringindexer", "Smoking Status_stringindexer", "Alcohol Intake_stringindexer", "Physical Activity_stringindexer", "Dietary Habits_stringindexer", "Blood Pressure Levels_stringindexer", "Cholesterol Levels_stringindexer", "Symptoms_stringindexer", "Diagnosis_stringindexer"], "Family History of Stroke_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

